In [1]:
import torchreid
import torch

# HyperParams
BATCH_SIZE = 128
NUM_INSTANCES = 4
BATCH_SIZE_TEST = 128
TRANSFORMS = ['random_flip', 'random_crop', 'norm_mean', 'norm_std', 'color_jitter', 'random_patch']  # 'random_erase'
SEQ_LEN = 2
MODEL_NAME = 'Vrt2_resnet50'
PRETRAIN = True
LOSS_TYPE = 'oim'
OPTIM = 'sgd'
INIT_LR = 0.0
LR_SCHEDULAR = 'cyclic'
LR_SCHEDULAR_STEPSIZE = 157 * 2
MAX_LR = 0.005   #25  # 25
BASE_LR = 0.005
WEIGHT_OIMLOSS = 0.005
WEIGHT_CELOSS = 1.0
WEIGHT_VERLOSS = 5000.0 # 太高模型就会为了去verify而破坏了representation learning
LABEL_SMOOTH = True
LOG_DIR = '/data/PycharmProjects/deep-person-reid/log/vrt2/exp0/0'
MAX_EPOCH = 400
EVAL_FREQ = 4

# Load data manager
datamanager = torchreid.data.VideoDataManager(
    root='/data/reid-data',
    sources='mars',
    targets='mars',
    height=224,  # 256, 128
    width=224,
    batch_size_train=BATCH_SIZE,
    batch_size_test=BATCH_SIZE_TEST,
    transforms=TRANSFORMS,
    norm_mean=[0.4058, 0.3852, 0.3730],
    norm_std=[0.2049, 0.1984, 0.1956],
    seq_len=SEQ_LEN,
    train_sampler='RandomIdentitySampler',
    num_instances=NUM_INSTANCES,
)


# Build model, optimizer and lr_scheduler
model = torchreid.models.build_model(
    name=MODEL_NAME,
    num_classes=datamanager.num_train_pids,
    loss=LOSS_TYPE,
    pretrained=PRETRAIN
)

model = model.cuda()


torchreid.utils.load_pretrained_weights(model, '/data/PycharmProjects/deep-person-reid/log/vrt2/exp0/0/model/model.pth.tar-8')


Building train transforms ...
+ resize to 224x224
+ random flip
+ random crop (enlarge to 252x252 and crop 224x224)
+ random patch
+ color jitter
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.4058, 0.3852, 0.373], std=[0.2049, 0.1984, 0.1956])
Building test transforms ...
+ resize to 224x224
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.4058, 0.3852, 0.373], std=[0.2049, 0.1984, 0.1956])
=> Loading train (source) dataset
=> Loaded Mars
  -------------------------------------------
  subset   | # ids | # tracklets | # cameras
  -------------------------------------------
  train    |   625 |        8298 |         6
  query    |   626 |        1980 |         6
  gallery  |   621 |        9330 |         6
  -------------------------------------------
=> Loading test (target) dataset
=> Loaded Mars
  -------------------------------------------
  subset   | # ids | # tracklets | # cameras
  -------------------------------------------
  train    |   625 |        829

In [4]:
for n, p in model.named_parameters():
    print(n, p)

backbone.conv1.weight Parameter containing:
tensor([[[[ 1.5775e-02,  1.6229e-02, -1.3112e-02,  ..., -3.7708e-02,
           -3.9400e-02, -6.5859e-02],
          [ 6.7081e-03,  7.8227e-03,  1.7175e-02,  ...,  5.3500e-03,
           -1.7034e-02, -3.3773e-02],
          [ 2.4860e-02,  2.5720e-02,  1.8861e-02,  ...,  1.0511e-01,
            6.5815e-02,  5.5382e-02],
          ...,
          [ 1.5542e-03,  2.9207e-02, -7.7957e-03,  ..., -1.2259e-01,
           -7.2768e-02,  9.8181e-03],
          [ 6.5179e-03,  4.9435e-02,  6.3207e-02,  ...,  2.6215e-02,
           -3.0655e-02, -1.3336e-02],
          [-7.6368e-02, -3.0076e-02, -1.6024e-02,  ...,  3.6134e-02,
            2.3722e-02,  3.1898e-03]],

         [[-1.7733e-02,  1.1112e-02,  2.3792e-02,  ...,  5.4047e-02,
            4.5257e-02, -6.6197e-03],
          [-7.1878e-03,  1.8686e-02,  6.7518e-02,  ...,  1.5858e-01,
            1.4613e-01,  1.2099e-01],
          [-4.4617e-02, -7.4395e-02, -8.7073e-02,  ...,  1.2143e-01,
            1.

backbone.layer2.3.bn1.bias Parameter containing:
tensor([-0.1183, -0.0079, -0.0985,  0.0467, -0.0874, -0.0412,  0.0248,  0.0200,
         0.0522, -0.1115,  0.0292,  0.0166,  0.0564, -0.1096,  0.1048, -0.1086,
        -0.0512, -0.0948, -0.1599, -0.0610,  0.0266,  0.0203, -0.0843, -0.1023,
        -0.0094, -0.0225, -0.0467, -0.0405,  0.0942, -0.1253, -0.0652, -0.0412,
        -0.0011, -0.0932,  0.0284,  0.0165, -0.0110, -0.0818,  0.0221, -0.1525,
         0.0377, -0.2135, -0.0522,  0.0753, -0.0553, -0.0452,  0.0415, -0.0335,
        -0.0195, -0.0823, -0.0346, -0.0132, -0.0896,  0.0174, -0.0099, -0.0097,
         0.0041,  0.0561, -0.1043,  0.0777, -0.0569,  0.0116, -0.1257,  0.0498,
         0.0614, -0.0378, -0.0439,  0.0092, -0.1234, -0.1058, -0.0261,  0.0086,
        -0.1003,  0.0207, -0.0996, -0.0132,  0.0344,  0.0608, -0.0401, -0.1154,
         0.1102,  0.0579,  0.0929,  0.0340,  0.0329,  0.0340, -0.0375, -0.0199,
         0.0219,  0.0347,  0.0209, -0.1169, -0.0008, -0.0004,  0.0452, 

backbone.layer4.0.bn3.weight Parameter containing:
tensor([0.3102, 0.4200, 0.3931,  ..., 0.3166, 0.3601, 0.3111], device='cuda:0',
       requires_grad=True)
backbone.layer4.0.bn3.bias Parameter containing:
tensor([-0.0585, -0.0900, -0.0668,  ..., -0.0586, -0.0631, -0.0649],
       device='cuda:0', requires_grad=True)
backbone.layer4.0.downsample.0.weight Parameter containing:
tensor([[[[-0.0011]],

         [[-0.0043]],

         [[ 0.0019]],

         ...,

         [[-0.0048]],

         [[-0.0080]],

         [[ 0.0034]]],


        [[[ 0.0010]],

         [[ 0.0035]],

         [[ 0.0022]],

         ...,

         [[-0.0037]],

         [[-0.0056]],

         [[-0.0023]]],


        [[[-0.0093]],

         [[-0.0071]],

         [[ 0.0012]],

         ...,

         [[-0.0154]],

         [[-0.0029]],

         [[ 0.0002]]],


        ...,


        [[[-0.0028]],

         [[ 0.0120]],

         [[ 0.0056]],

         ...,

         [[ 0.0038]],

         [[ 0.0124]],

         [